In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd #conda install geopandas
import matplotlib.pyplot as plt

from haversine import haversine

## Data

In [2]:
PATH = 'C:/Users/Francisco Sarabia/Documents/0.datasets'
STS = pd.read_csv(PATH+'/CAT-Estaciones_csv.csv', delimiter = '|')
#STS.head()
stations = STS[['ID', 'Name', 'Lat', 'Lon']]
stations['LatLon'] = list(zip(stations.Lat, stations.Lon)) # in tuples!
#stations.head()

C:\Users\Francisco Sarabia\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [5]:
PTS = pd.read_csv(PATH+'/CAT-Hospitales-SEMOVI_csv.csv', delimiter = "|")
#PTS.head()
points = PTS[['ID', 'Name', 'Lat', 'Lon']]
points['LatLon'] = list(zip(points.Lat, points.Lon)) # in tuples!
#points.head()

C:\Users\Francisco Sarabia\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


## Haversine

In [75]:
dist = []

for i in points['LatLon']:
    for j in stations['LatLon']:
        dist.append(str(haversine(i,j,unit='km')))

#print(dist)

In [76]:
len(dist), type(dist)

(29760, list)

In [77]:
distdf = pd.DataFrame(dist)
distdf.to_csv('distdf.csv')
distdf.head()

,0
0,2.0191679899769865
1,1.940750117217219
2,2.9087157326082895
3,2.233239227743033
4,2.2324676404853667


## Lavels

In [153]:
pairs = []

for i in stations['ID']:
    for j in points['ID']:
        pairs.append(str(j) + ";" + str(i))
#print(pairs)

In [154]:
len(pairs), type(pairs)

(29760, list)

In [155]:
pairsdf = pd.DataFrame(pairs)
pairsdf.to_csv('pairsdf.csv')
pairsdf.head()

,0
0,H1;1
1,H1;1
2,H2;1
3,H3;1
4,H4;1


## Laveling

In [80]:
length = len(dist)
distances = pd.DataFrame({'ids':pairs, 'dist':dist})
distances.head()

,ids,dist
0,H1;1,2.0191679899769865
1,H1;1,1.940750117217219
2,H2;1,2.9087157326082895
3,H3;1,2.233239227743033
4,H4;1,2.2324676404853667


In [82]:
distances.dtypes, type(distances), distances.shape

(ids     object
 dist    object
 dtype: object, pandas.core.frame.DataFrame, (29760, 2))

In [83]:
distances['dist'] = distances.dist.astype(float)
distances['dist'].head()

0    2.019168
1    1.940750
2    2.908716
3    2.233239
4    2.232468
Name: dist, dtype: float64

In [84]:
distances.dtypes, type(distances), distances.shape

(ids      object
 dist    float64
 dtype: object, pandas.core.frame.DataFrame, (29760, 2))

In [85]:
distances.to_csv('distances.csv')

In [94]:
nearest = distances.query('dist < .3')
#nearest = distances[(distances.dist < 3) & (distances.dist > 2)]
nearest.head()

,ids,dist
478,H44;8,0.288758
479,H45;8,0.029736
648,H28;11,0.255462
654,H34;11,0.220508
658,H38;11,0.220400


In [93]:
nearest.shape

(1903, 2)

In [96]:
nearest.to_csv('nearest.csv')

## METODO 2

In [3]:
#stations.head()
stations.shape

(480, 5)

In [6]:
#points.head()
points.shape

(62, 5)

In [7]:
df1 = stations[['ID','Name', 'Lat', 'Lon', 'LatLon']].rename(columns={'ID':'id1', 'Name':'name1', 'Lat':'lat1','Lon':'lon1', 'LatLon':'LatLon1'})
df2 = points[['ID','Name', 'Lat', 'Lon', 'LatLon']].rename(columns={'ID':'id2', 'Name':'name2', 'Lat':'lat2','Lon':'lon2', 'LatLon':'LatLon2'})

In [8]:
df1.head()

,id1,name1,lat1,lon1,LatLon1
0,1,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,"(19.433571399999998, -99.16780909999999)"
1,2,2 RIO GUADALQUIVIR-RIO BALSAS,19.431386,-99.171695,"(19.431386, -99.171695)"
2,3,3 REFORMA-INSURGENTES,19.431655,-99.158668,"(19.431655, -99.158668)"
3,4,4 RIO NILO-RIO PANUCO,19.428309,-99.171713,"(19.428309, -99.171713)"
4,5,5 RIO PANUCO-RIO TIBER,19.429720,-99.169363,"(19.42972, -99.169363)"


In [9]:
df2.head()

,id2,name2,lat2,lon2,LatLon2
0,H1,Centro de Salud México - España,19.445945,-99.181903,"(19.445945000000002, -99.181903)"
1,H1,Centro Médico Quirúrgico Escandón,19.402750,-99.172515,"(19.40275, -99.172515)"
2,H2,Centro Médico Siglo XXI,19.406922,-99.154821,"(19.406922, -99.154821)"
3,H3,Centro Médico Tíber,19.432640,-99.170896,"(19.43264, -99.170896)"
4,H4,Clínica 38 IMSS,19.416483,-99.170102,"(19.416483, -99.170102)"


In [10]:
df1.shape, df2.shape

((480, 5), (62, 5))

In [11]:
import math

#def distance(origin, destination):
#    lat1, lon1 = origin
#    lat2, lon2 = destination
#    radius = 6371 # km

#    dlat = math.radians(lat2-lat1)
#    dlon = math.radians(lon2-lon1)
#    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
#        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
#    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
#    d = radius * c

#    return d

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    d = math.sqrt((lat1-lat2)**2 + (lon1-lon2)**2)
    
    return d

In [12]:
origin = (2,-3)
lat1, lon1 = origin
print(lat1, lon1)

2 -3


In [13]:
dist = []

for i in df2['LatLon2']:
    for j in df1['LatLon1']:
        dist.append(str(distance(i,j)))
#        distance(i,j)
#print(dist)

In [14]:
len(dist), type(dist)

(29760, list)

In [20]:
pairs = []

for i in df2['id2']:
    for j in df1['id1']:
        pairs.append(str(j) + ";" + str(i))
#print(pairs)

In [21]:
distdf = pd.DataFrame(dist)
distdf.to_csv('distdf.csv')
#distdf.head()

pairsdf = pd.DataFrame(pairs)
pairsdf.to_csv('pairsdf.csv')
#pairsdf.head()

In [22]:
length = len(dist)
distances2 = pd.DataFrame({'ids':pairs, 'dist':dist})
distances2.head()

,ids,dist
0,1;H1,0.018754839220069566
1,2;H1,0.017781106405400527
2,3;H1,0.027277634153277636
3,4;H1,0.02036822515587168
4,5;H1,0.02050615090649884


In [19]:
distances2['dist'] = distances2.dist.astype(float)
distances2['dist'].head()

distances2.to_csv('distances3.csv')